# Sentence Window Retrieval

In [1]:
from dotenv import load_dotenv
import os
import nest_asyncio
nest_asyncio.apply()

load_dotenv()
api_key=os.getenv("MISTRAL_API_KEY")

In [2]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.llms.mistralai import MistralAI
from llama_index.core import Settings

#Setting up model
embed_model= MistralAIEmbedding(api_key=api_key)
llm= MistralAI(api_key=api_key)
Settings.llm= llm
Settings.embed_model=embed_model

In [3]:
#Loading document
docs=SimpleDirectoryReader(input_files=["../data/master-direction-rbi/Digital Payment Security Controls.txt"]).load_data()

In [4]:
#Splitting Chapter
import re

def split_by_chapters(text):
    # Step 1: Split text into chapters by `#`
    chapters = re.split(r"(# .+)", text)  # Match chapter titles starting with `# `
    chapter_chunks = []

    for i in range(1, len(chapters), 2):  # Iterate over matched chapter titles and their content
        chapter_title = chapters[i].strip()
        chapter_content = chapters[i + 1].strip()
        # chapter_chunks.append(f"{chapter_title}\n{chapter_content}")
        chapter_chunks.append({
            "title": chapter_title,
            "content": chapter_content
        })


    return chapter_chunks

chapter_chunks= split_by_chapters(docs[0].text)

In [5]:
#Checking split
for i, chunk in enumerate(chapter_chunks[:3]):
        print(f"Chunk: {i+1}")
        print(chunk)

Chunk: 1
{'title': '# INTRODUCTION', 'content': 'In exercise of the powers conferred by the Banking Regulation Act, 1949, the Reserve Bank of India Act, 1934 and Payment and Settlement Systems Act, 2007, the Reserve Bank, being satisfied that it is necessary and expedient in the public interest so to do, hereby, issues the directions hereinafter specified.'}
Chunk: 2
{'title': '# CHAPTER – I: PRELIMINARY', 'content': '1 Short Title and Commencement\n\na) These directions shall be called the Reserve Bank of India (Digital Payment Security Controls) directions, 2021.\n\nb) These directions shall come into effect six months from the day they are placed on the official website of the Reserve Bank of India (RBI). However, in respect of instructions already issued either by Department of Payment and Settlement Systems (DPSS), Department of Regulation (DoR) or Department of Supervision (DoS) of RBI including those to select Regulated Entities (REs), by way of circular or advisory, the timelin

In [6]:
#Importing library for splitting
from llama_index.core import Document
import nltk

def sentence_split(chapter_chunks):
    nodes= []
    for section in chapter_chunks:
        # Preprocess the text to replace multiple consecutive newline characters with a single newline character
        content = re.sub(r'\n{2,}', '\n', section["content"])
        # Preprocess the text to replace newline characters with a space
        content = re.sub(r'(?<!\.)\n', '.\n', content)
        sentences=nltk.sent_tokenize(content)
        for sentence in sentences:
            node= Document(text=sentence,
                        metadata={
                            "title": section["title"]
                        })
            nodes.append(node)
    return nodes

sentences= sentence_split(chapter_chunks)

In [7]:
#Checking sentences
for i in range(len(sentences[:5])):
    print(f"\nSentence {i}:")
    print(sentences[i].get_content())
    print("Title: {}".format(sentences[i].metadata.get("title")))


Sentence 0:
In exercise of the powers conferred by the Banking Regulation Act, 1949, the Reserve Bank of India Act, 1934 and Payment and Settlement Systems Act, 2007, the Reserve Bank, being satisfied that it is necessary and expedient in the public interest so to do, hereby, issues the directions hereinafter specified.
Title: # INTRODUCTION

Sentence 1:
1 Short Title and Commencement.
Title: # CHAPTER – I: PRELIMINARY

Sentence 2:
a) These directions shall be called the Reserve Bank of India (Digital Payment Security Controls) directions, 2021.
b) These directions shall come into effect six months from the day they are placed on the official website of the Reserve Bank of India (RBI).
Title: # CHAPTER – I: PRELIMINARY

Sentence 3:
However, in respect of instructions already issued either by Department of Payment and Settlement Systems (DPSS), Department of Regulation (DoR) or Department of Supervision (DoS) of RBI including those to select Regulated Entities (REs), by way of circular

In [8]:
#Building vectore store index
index_directory='../data/master-direction-rbi/index'
if not os.path.exists(index_directory+'/md-rbi'):
    index=VectorStoreIndex.from_documents(sentences)
    index.storage_context.persist(persist_dir=index_directory+'/md-rbi')
else:
    index= load_index_from_storage(StorageContext.from_defaults(persist_dir=index_directory+'/md-rbi'))

In [9]:
#Custom class for retriving unique chunk
from typing import List
from llama_index.core.indices.vector_store.retrievers import VectorIndexRetriever
from llama_index.core.schema import NodeWithScore, QueryBundle

class AllNodesUniqueTitleVectorIndexRetriever(VectorIndexRetriever):
    def __init__(self, *args, **kwargs):
        # Set similarity_top_k to a very large number to retrieve all nodes
        kwargs['similarity_top_k'] = 10000000000
        super().__init__(*args, **kwargs)
        self.chapter_chunks = kwargs.get('chapter_chunks', [])

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        # Call the parent class's _retrieve method to get all the initial results
        initial_results = super()._retrieve(query_bundle)

        # Filter out duplicate documents based on the "title" metadata
        seen_titles = set()
        unique_results = []
        for result in initial_results:
            title = result.node.metadata.get("title", None)
            if title and title not in seen_titles:
                seen_titles.add(title)
                unique_results.append(result)

        # Sort the unique results by score (if available) and select the top 5
        unique_results.sort(key=lambda x: x.score if x.score is not None else float('inf'), reverse=True)
        top_5_unique_results = unique_results[:5]

        # Replace text with window content after retrieval
        top_5_unique_results = self._replace_text_with_window_content(top_5_unique_results)

        return top_5_unique_results

    def _replace_text_with_window_content(self, results: List[NodeWithScore]) -> List[NodeWithScore]:
        for result in results:
            title = result.node.metadata.get("title")
            window_content = next((chunk["content"] for chunk in self.chapter_chunks if chunk["title"] == title), result.node.text)
            result.node.text = window_content
        return results

In [10]:
# Create an instance of the custom retriever
retriever = AllNodesUniqueTitleVectorIndexRetriever(index, chapter_chunks=chapter_chunks)

In [11]:
#Creating query engine
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine

query_engine= RetrieverQueryEngine.from_args(retriever)

## Initialize Relevance Evaluator

In [12]:
from custom.faithfulness import FaithfulnessEvaluator
from custom.answer_relevance import AnswerRelevancyEvaluator
from custom.context_relevance import  ContextRelevancyEvaluator

answer_relevance_evaluator = AnswerRelevancyEvaluator(llm=llm)
faithfulness_evaluator= FaithfulnessEvaluator(llm=llm)
context_relevance_evaluator= ContextRelevancyEvaluator(llm=llm)

In [13]:
from llama_index.core.evaluation import EvaluationResult
from llama_index.core import Response
import pandas as pd
pd.set_option("display.max_colwidth", 0)


def display_eval_df(
    query: str, response: Response, answer_relevance_eval_result: EvaluationResult, faithfulness_eval_result: EvaluationResult, context_relevance_eval_result: EvaluationResult
) -> None:
    
    #For better display, retriving first 200 characters of each source document
    contexts=[source[:200] for source in [node.get_content()[:200] for node in response.source_nodes]]
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response),
            "Source": "\n\n".join([f"##Source {i+1}:\n{context}....." for i, context in enumerate(contexts)]),
            "Answer Relevance Reasoning": answer_relevance_eval_result.feedback,
            "Faithfulness Reasoning": faithfulness_eval_result.feedback,
            "Context Relevance Reasoning": context_relevance_eval_result.feedback
        },
        index=[0],
    )
    return eval_df

In [15]:
questions=["What controls are suggested for card payments security?",
           "What controls/provisions are suggested for security of internet banking?",
           "What provisions related to multi factor authentication are mentioned in documents?",
           "What provisions related to encryption are outlined in the documents?"]
eval_df=pd.DataFrame()
for query_str in questions:
    response = query_engine.query(query_str)

    relevance_eval_result = answer_relevance_evaluator.evaluate_response(query=query_str, response=response)

    faithfulness_eval_result = faithfulness_evaluator.evaluate_response(response=response)

    context_relevance_eval_result= context_relevance_evaluator.evaluate_response(query=query_str, response=response)
    
    return_df=display_eval_df(query_str, response, relevance_eval_result, faithfulness_eval_result, context_relevance_eval_result)
    if eval_df.empty:
        eval_df=return_df
    else:
        eval_df=pd.concat([eval_df,return_df], ignore_index=True)

eval_df = eval_df.style.set_properties(
    **{
        "inline-size": "600px",
        "overflow-wrap": "break-word",
        "text-align": "left",
        "vertical-align": "top",  # Top align the text
    },
    subset=["Response", "Source", "Answer Relevance Reasoning", "Faithfulness Reasoning", "Context Relevance Reasoning"]
).set_properties(
    **{
        "text-align": "left",
        "white-space": "pre-wrap",
        "vertical-align": "top",  # Top align the text
    },
    subset=["Query"]
)
display(eval_df)

,Query,Response,Source,Answer Relevance Reasoning,Faithfulness Reasoning,Context Relevance Reasoning
0,What controls are suggested for card payments security?,"For card payments security, it is suggested to follow various payment card standards, including but not limited to PCI-PIN, PCI-PTS, PCI-HSM, and PCI-P2PE. Additionally, terminals at merchants should be validated against the PCI-P2PE program, and acquirers should secure their card payment infrastructure. Hardware Security Modules (HSMs) should have logging enabled, be tamper-proof, and access should be controlled through Access Control Lists (ACLs). Security measures for ATMs include implementing BIOS passwords, disabling USB ports, applying the latest patches, and using anti-skimming and whitelisting solutions. Robust surveillance and monitoring of card transactions, especially overseas cash withdrawals, should be in place, with transaction limits set at the card, BIN, and RE levels. Card data scanning tools should be tested in a safe environment and used only within the RE's premises.","##Source 1: Governance and Management of Security Risks 4 REs shall formulate a policy for digital payment products and services with the approval of their Board. The contours of the policy, while discussing the..... ##Source 2: CARD PAYMENTS SECURITY In addition to the controls prescribed in Chapter II, the following instructions are applicable to the REs offering/ intending to issue cards (credit/ debit/ prepaid) (physical..... ##Source 3: MOBILE PAYMENTS APPLICATION SECURITY CONTROLS In addition to the controls prescribed in Chapter II, the following instructions are applicable to the REs offering/ intending to offer mobile banking/ m..... ##Source 4: INTERNET BANKING SECURITY CONTROLS In addition to the controls prescribed in Chapter II, the following instructions are applicable to REs offering/ intending to offer internet banking facility to the..... ##Source 5: 1 Short Title and Commencement a) These directions shall be called the Reserve Bank of India (Digital Payment Security Controls) directions, 2021. b) These directions shall come into effect six mont.....","Feedback: The response provides a comprehensive list of controls suggested for card payments security, addressing various aspects such as adherence to PCI standards, terminal validation, hardware security, ATM security measures, transaction monitoring, and data scanning tools. It directly answers the query by offering specific and detailed recommendations. EVALUATION SCORE: 1","Feedback: The response is highly faithful to the given context. It accurately reflects the various security measures mentioned in the context, including the specific PCI standards, the need for validation against the PCI-P2PE program, the importance of securing hardware security modules (HSMs), and the implementation of security measures for ATMs. The response also correctly mentions the need for robust surveillance and monitoring of card transactions, setting transaction limits, and the proper use of card data scanning tools. All these points are directly supported by the information provided in the context. EVALUATION SCORE: 1.","Feedback: The retrieved context is highly relevant to the query. It provides extensive details on various security controls and measures that should be implemented for digital payment products and services, including specific sections on card payments security, mobile payments application security controls, and internet banking security controls. The context discusses necessary controls to protect the confidentiality of customer data, integrity of data and processes, availability of requisite infrastructure, security aspects of digital payment products, risk management, fraud risk management, and more. These details are directly applicable to answering the query about suggested controls for card payments security. EVALUATION SCORE: 1"
1,What controls/provisions are suggested for security of internet banking?,"To enhance

### Saving result in excel

In [16]:
#eval_df.to_excel("../data/master-direction-rbi/results.xlsx", index=False)

In [20]:
#Text-align: justify
questions=["What controls are suggested for card payments security?",
           "What controls/provisions are suggested for security of internet banking?",
           "What provisions related to multi factor authentication are mentioned in documents?",
           "What provisions related to encryption are outlined in the documents?"]

eval_df=pd.DataFrame()
for query_str in questions:
    response_vector = query_engine.query(query_str)
    relevance_eval_result = answer_relevance_evaluator.evaluate_response(
    query=query_str, response=response_vector)
    faithfulness_eval_result = faithfulness_evaluator.evaluate_response(
    query=query_str, response=response_vector)
    context_relevance_eval_result= context_relevance_evaluator.evaluate_response(
    query=query_str, response=response_vector)
    return_df=display_eval_df(query_str, response_vector, relevance_eval_result, faithfulness_eval_result, context_relevance_eval_result)
    if eval_df.empty:
        eval_df=return_df
    else:
        eval_df=pd.concat([eval_df,return_df], ignore_index=True)

eval_df = eval_df.style.set_properties(
    **{
        "inline-size": "600px",
        "overflow-wrap": "break-word",
        "text-align": "justify",
        "vertical-align": "top",  # Top align the text
    },
    subset=["Response", "Source", "Answer Relevance Reasoning", "Faithfulness Reasoning", "Context Relevance Reasoning"]
).set_properties(
    **{
        "text-align": "justify",
        "white-space": "pre-wrap",
        "vertical-align": "top",  # Top align the text
    },
    subset=["Query"]
)
display(eval_df)

,Query,Response,Source,Answer Relevance Reasoning,Faithfulness Reasoning,Context Relevance Reasoning
0,What controls are suggested for card payments security?,"For card payments security, it is suggested to follow various payment card standards, including PCI-PIN, PCI-PTS, PCI-HSM, and PCI-P2PE, to ensure comprehensive security. Additionally, terminals at merchants should be validated against the PCI-P2PE program, and acquirers should secure their card payment infrastructure. Hardware Security Modules (HSMs) should have logging enabled, be tamper-proof, and access should be controlled through Access Control Lists (ACLs). Security measures for ATMs include implementing BIOS passwords, disabling USB ports, applying the latest patches, and using anti-skimming and whitelisting solutions. Robust surveillance and monitoring of card transactions, especially overseas cash withdrawals, should be in place, with transaction limits set at the card, BIN, and RE levels. Card data scanning tools should be tested in a safe environment and used only within the RE's premises.","##Source 1: Governance and Management of Security Risks 4 REs shall formulate a policy for digital payment products and services with the approval of their Board. The contours of the policy, while discussing the..... ##Source 2: CARD PAYMENTS SECURITY In addition to the controls prescribed in Chapter II, the following instructions are applicable to the REs offering/ intending to issue cards (credit/ debit/ prepaid) (physical..... ##Source 3: MOBILE PAYMENTS APPLICATION SECURITY CONTROLS In addition to the controls prescribed in Chapter II, the following instructions are applicable to the REs offering/ intending to offer mobile banking/ m..... ##Source 4: INTERNET BANKING SECURITY CONTROLS In addition to the controls prescribed in Chapter II, the following instructions are applicable to REs offering/ intending to offer internet banking facility to the..... ##Source 5: 1 Short Title and Commencement a) These directions shall be called the Reserve Bank of India (Digital Payment Security Controls) directions, 2021. b) These directions shall come into effect six mont.....","Feedback: The response provides a comprehensive list of controls suggested for card payments security, directly addressing the query. It mentions specific standards like PCI-PIN, PCI-PTS, PCI-HSM, and PCI-P2PE, as well as practical measures such as validating terminals, securing infrastructure, and implementing various security protocols for ATMs. The response also highlights the importance of surveillance, monitoring, and setting transaction limits. Overall, the response is highly relevant and sufficiently answers the query. EVALUATION SCORE: 1","Feedback: The response is largely faithful to the context provided. It accurately mentions various PCI standards for card payments security, the need for validating terminals against the PCI-P2PE program, and the importance of securing card payment infrastructure. It also correctly highlights the security measures for ATMs, such as implementing BIOS passwords, disabling USB ports, applying patches, and using anti-skimming and whitelisting solutions. The response also includes the need for robust surveillance and monitoring of card transactions, setting transaction limits, and the proper use of card data scanning tools. However, the context does not explicitly mention ""Access Control Lists (ACLs)"" for controlling access to HSMs, which is included in the response. This minor discrepancy slightly affects the overall faithfulness of the response to the context. EVALUATION SCORE: 0.9","Feedback: The retrieved context is highly relevant to the query. It provides extensive details on various security controls and measures that should be implemented for digital payment products, including card payments. The context covers a wide range of security aspects such as encryption, authentication, risk management, fraud detection, and compliance with standards like PCI-DSS. Th